In [56]:
#!g1.1
%pip install --upgrade gdown
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14763 sha256=693c20eef6e6cd33e041e0c1a1ca26787623fe75be220c625ad58b8215981b9f
  Stored in directory: /tmp/xdg_cache/pip/wheels/7b/7b/5d/656f46cd6889e4c93977be9586901d0adc1271b2d876c84c96
Successfully built gdown
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [65]:
#!g1.1
!cp -r HiAGM/data_modules .
!cp -r HiAGM/models .
!cp -r HiAGM/helper .

In [318]:
#!g1.1
def apply_to_dict_values(dict, f):
    for key, value in dict.items():
        dict[key] = f(value)

In [319]:
#!g1.1
BERT_TYPE = 'bert-base-uncased'

In [320]:
#!g1.1
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig

class RCVDataset(Dataset):
    """RCV dataset."""

    def __init__(self, path):
        self.path = path
        self.data = pd.read_csv(self.path, sep='\t', header=None)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_TYPE)

    def __len__(self):
        return self.data.shape[0]

    @staticmethod
    def target_to_tensor(target):
        return torch.tensor([float(label) for label in target])

    def __getitem__(self, idx):
        data = self.tokenizer(self.data.iloc[idx, 1], return_tensors="pt", max_length=512, padding="max_length", truncation=True) # max_len=512 !DocBERT
        apply_to_dict_values(data, lambda x: x.flatten())
        return data, RCVDataset.target_to_tensor(self.data.iloc[idx, 0])

In [321]:
#!g1.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [322]:
#!g1.1
train_dataset = RCVDataset('./train_docbert.wos')
val_dataset = RCVDataset('./valid_docbert.wos')
test_dataset = RCVDataset('./test_docbert.wos')

In [323]:
#!g1.1
%pip install wandb
from wandb_writer import WandbWriter

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [324]:
#!g1.1
BATCH_SIZE = 16 # !DocBERT
N_CLASSES = test_dataset[0][1].shape[0]
BATCH_SIZE, N_CLASSES

(16, 141)

In [325]:
#!g1.1
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [326]:
#!g1.1
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [327]:
#!g1.1
from models.structure_model.structure_encoder import StructureEncoder
from helper.configure import Configure

In [337]:
#!g1.1
config = Configure(config_json_file='HiAGM/config/rcv1-v2-Copy2.json')

INFO:  CONFIGURE: {'dataset': 'wos', 'data_dir': 'HiAGM/data', 'train_file': 'wos_train.json', 'val_file': 'wos_val.json', 'test_file': 'wos_test.json', 'prob_json': 'wos_prob.json', 'hierarchy': 'wos.taxnomy'}
INFO:  CONFIGURE: {'dir': 'vocab', 'vocab_dict': 'word.dict', 'max_token_vocab': 60000, 'label_dict': 'label.dict'}
INFO:  CONFIGURE: {'dimension': 300, 'type': 'pretrain', 'pretrained_file': 'glove.6B/glove.6B.300d.txt', 'dropout': 0.5, 'init_type': 'uniform'}
INFO:  CONFIGURE: {'dimension': 768, 'type': 'random', 'dropout': 0.5, 'init_type': 'kaiming_uniform'}
INFO:  CONFIGURE: {'token': <helper.configure.Configure object at 0x7fc3c98f7370>, 'label': <helper.configure.Configure object at 0x7fc3c98f7610>}
INFO:  CONFIGURE: {'bidirectional': True, 'num_layers': 1, 'type': 'GRU', 'hidden_dimension': 64, 'dropout': 0.1}
INFO:  CONFIGURE: {'kernel_size': [2, 3, 4], 'num_kernel': 100}
INFO:  CONFIGURE: {'max_length': 256, 'RNN': <helper.configure.Configure object at 0x7fc3c98f79a0>,

In [338]:
#!g1.1
import torch.nn as nn

class SEDocBert(nn.Module):
    def __init__(self, bert_model, n_classes, config, vocab, freeze_bert_weights=False):
        super(SEDocBert, self).__init__()
        self.bert_model = bert_model
        if freeze_bert_weights: # == false !DocBERT
            for param in self.bert_model.parameters():
                param.requires_grad = False
        self.label_map = vocab.v2i['label']
        self.transformation = nn.Linear(bert_model.pooler.dense.out_features,
                                        len(self.label_map) * config.model.linear_transformation.node_dimension)
        self.transformation_dropout = nn.Dropout(p=0.5)
        self.linear = nn.Linear(len(self.label_map) * config.embedding.label.dimension,
                                len(self.label_map))
        self.graph_model = StructureEncoder(config=config,
                                            label_map=self.label_map,
                                            device='cuda',
                                            graph_model_type=config.structure_encoder.type)
        self.dropout = nn.Dropout(p=0.5)
        self.config = config

    def forward(self, inputs):
        bert_output = self.bert_model(**inputs)
        
        bert_output = bert_output.pooler_output
        
        bert_output = self.transformation_dropout(self.transformation(bert_output))
        
        bert_output = bert_output.view(bert_output.shape[0],
                                       len(self.label_map),
                                       self.config.model.linear_transformation.node_dimension)
        
        label_wise_bert_output = self.graph_model(bert_output)
        logits = self.dropout(self.linear(label_wise_bert_output.view(label_wise_bert_output.shape[0], -1)))
        return logits

In [339]:
#!g1.1
from data_modules.vocab import Vocab
vcb = Vocab(config, min_freq=5, max_size=50000)

400000it [01:07, 5937.08it/s]
30070it [00:03, 9746.23it/s]
7518it [00:00, 8367.99it/s]
9397it [00:01, 8678.60it/s]


INFO:  Building Vocabulary....
INFO:  Generating Vocabulary from Corpus...
INFO:  Loading vocabulary from pretrained embedding...
INFO:  Loading TRAIN subset...
INFO:  Loading VAL subset...
INFO:  Loading TEST subset...
INFO:  Vocabulary of token 465077
INFO:  Vocabulary of label 141
INFO:  Shrinking Vocabulary...
INFO:  Shrinking Vocabulary of tokens: 50000
INFO:  Vocabulary of token with the size of 50002
INFO:  Save Vocabulary in vocab/word.dict
INFO:  Vocabulary of label with the size of 141
INFO:  Save Vocabulary in vocab/label.dict


In [340]:
#!g1.1
import torch.optim as optim

LEARNING_RATE = 2e-5

In [341]:
#!g1.1
from sklearn.metrics import classification_report

def test_model(
        model,
        test_dataloader,
        criterion,
        threshold,
        wandb_writer):

    model.eval()

    with torch.no_grad():
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(test_dataloader, f"Test epoch#{0}", leave=False):
            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets[:, 1:].to(device)
            logits = model(batch)[:, 1:]
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > threshold).type(torch.DoubleTensor).to('cpu').tolist())
#             loss = criterion(logits, targets)
#             total_loss += loss.item()

#         print("Validation loss", total_loss / len(val_dataloader))
        wandb_writer.add_scalar("Test F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        wandb_writer.add_scalar("Test F1 (macro)",(f1_score(all_targets, all_preds, average='macro')))
        wandb_writer.add_scalar("Test Hamming loss",(hamming_loss(all_targets, all_preds)))
        print("Test F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        print("Test F1 (macro)",(f1_score(all_targets, all_preds, average='macro')))
        print("Test Hamming loss",(hamming_loss(all_targets, all_preds)))

In [342]:
#!g1.1
import tqdm
from sklearn.metrics import f1_score, hamming_loss

def train_model(
        model,
        train_dataloader,
        val_dataloader,
        test_dataloader,
        optimizer,
        criterion,
        n_epochs,
        wandb_writer,
        wandb_iter_start = 0):

    wandb_iter = wandb_iter_start
    for epoch in range(1, n_epochs+1):
        model.train()
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(train_dataloader, f"Train epoch#{epoch}", leave=False):

            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets.to(device)
            logits = model(batch)
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
            optimizer.zero_grad()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            wandb_writer.set_step(wandb_iter)
            wandb_writer.add_scalar("Batch train loss", loss.item())
            wandb_iter += 1

        wandb_writer.add_scalar("Train loss", total_loss / len(train_dataloader))
        wandb_writer.add_scalar("Train F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        wandb_writer.add_scalar("Train F1 (macro)",(f1_score(all_targets, all_preds, average='macro')))
        wandb_writer.add_scalar("Train Hamming loss",(hamming_loss(all_targets, all_preds)))


        model.eval()

        with torch.no_grad():
            total_loss = 0.
            all_targets = []
            all_preds = []
            for batch, targets in tqdm.tqdm(val_dataloader, f"Val epoch#{epoch}", leave=False):
                apply_to_dict_values(batch, lambda x: x.to(device))
                targets = targets.to(device)
                logits = model(batch)
                all_targets.extend(targets.to('cpu').tolist())
                all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
                loss = criterion(logits, targets)
                total_loss += loss.item()
            
            wandb_writer.add_scalar("Validation loss", total_loss / len(val_dataloader))
            wandb_writer.add_scalar("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
            wandb_writer.add_scalar("Validation F1 (macro)",(f1_score(all_targets, all_preds, average='macro')))
            wandb_writer.add_scalar("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))
    
    test_model(model, test_dataloader, criterion, 0.5, wandb_writer)
    print(wandb_iter)

In [343]:
#!g1.1
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 2.003 Gb


In [344]:
#!g1.1
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats('cuda')
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 2.003 Gb


In [345]:
#!g1.1
model = SEDocBert(bert, N_CLASSES, config, vcb).to(device)
optimizer = optim.AdamW(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE, weight_decay=0.01)

In [346]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 15, WandbWriter("SEDocBERT"), wandb_iter_start=0)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_divisio

Test F1 (micro) 0.8574981204094615
Test F1 (macro) 0.8022815716681327
Test Hamming loss 0.0037458763435138875
28200


In [347]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=28200)

Batch train loss,▆▆██▅▆▂▆▅▇▅▆▅▅▅▄▄▂▅▆▄▆▄▅▆▅▄▃▄▅▅▅▄▁▂▃▅▇▃▃
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▁▅▆▆▇▇▇▇▇██████
Train F1 (micro),▁▅▆▆▇▇▇▇▇██████
Train Hamming loss,█▄▃▃▃▂▂▂▂▂▁▁▁▁▁
Train loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁
Validation F1 (macro),▁▅▆▇███████████
Validation F1 (micro),▁▅▆▇█████▇▇████
Validation Hamming loss,█▄▃▁▁▁▁▁▁▂▂▂▁▁▁


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


KeyboardInterrupt: 

In [ ]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=37600)

In [ ]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 1, WandbWriter("SEDocBERT"), wandb_iter_start=47000)

In [263]:
#!g1.1
start = 1880 * 14 + 1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 2, WandbWriter("SEDocBERT"), wandb_iter_start=start)

Batch train loss,▁▄▄▇▂▄▅▄▁▅▄▄▆▆▄▅▄▅▄█▃▆▄▂▄▅▆▄▃▅▇▆▆▄▄▃▆▅▄▆
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▁█
Train F1 (micro),▁█
Train Hamming loss,█▁
Train loss,█▁
Validation F1 (macro),▁█
Validation F1 (micro),▁█
Validation Hamming loss,█▁


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set

Test F1 (micro) 0.8520535248798007
Test F1 (macro) 0.79622033855567
Test Hamming loss 0.003882698125541586
30081


In [264]:
#!g1.1
start = 1880 * 16 + 1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 2, WandbWriter("SEDocBERT"), wandb_iter_start=start)

Batch train loss,▃▇▄▅▆▄▄▅▅▆▆▄▇▁▅▃▄▅▂▂▆▄▅▆▄▄▂▆█▅▅▄█▅▅▆▅▄▄▅
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),█▁
Train F1 (micro),█▁
Train Hamming loss,▁█
Train loss,▁█
Validation F1 (macro),▁█
Validation F1 (micro),▁█
Validation Hamming loss,█▁


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set

Test F1 (micro) 0.854943787835111
Test F1 (macro) 0.7998291133452152
Test Hamming loss 0.0038249289286854467
33841


In [265]:
#!g1.1
start = 1880 * 18 + 1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 2, WandbWriter("SEDocBERT"), wandb_iter_start=start)

Batch train loss,▃▅▄▃▄▃▄▅▆▇▄█▇▆▄▆▅▅▆▄▄▄▅▅▄▄▄▆▆▄▄▃▃▅▅▄▅▄▂▁
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▁█
Train F1 (micro),▁█
Train Hamming loss,█▁
Train loss,█▁
Validation F1 (macro),█▁
Validation F1 (micro),█▁
Validation Hamming loss,▁█


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set

Test F1 (micro) 0.8576551564578634
Test F1 (macro) 0.8025797215595936
Test Hamming loss 0.0037413156174462975
37601


In [266]:
#!g1.1
start = 1880 * 20 + 1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 1, WandbWriter("SEDocBERT"), wandb_iter_start=start)

Batch train loss,▄▄▃▁▃▂▃▆▄▄▃▇▄▄▁█▄▃▄▄▇▅▄▃▅▇▃▂▂▃▄▂▃▁█▅▄▃▂▅
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▁█
Train F1 (micro),▁█
Train Hamming loss,█▁
Train loss,█▁
Validation F1 (macro),▁█
Validation F1 (micro),▁█
Validation Hamming loss,█▁


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


Test F1 (micro) 0.8586671280675953
Test F1 (macro) 0.8035990148828246
Test Hamming loss 0.003725353076209733
39481


In [237]:
#!g1.1
from sklearn.metrics import classification_report

def test_model(
        model,
        val_dataloader,
        test_dataloader,
        criterion,
        threshold):

    model.eval()

    with torch.no_grad():
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(test_dataloader, f"Test epoch#{0}", leave=False):
            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets[:, 1:].to(device)
            logits = model(batch)[:, 1:]
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > threshold).type(torch.DoubleTensor).to('cpu').tolist())
#             loss = criterion(logits, targets)
#             total_loss += loss.item()

#         print("Validation loss", total_loss / len(val_dataloader))
        print("Test F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        print("Test F1 (macro)",(f1_score(all_targets, all_preds, average='macro')))
        print("Test Hamming loss",(hamming_loss(all_targets, all_preds)))

In [238]:
#!g1.1
test_model(model, val_dataloader, test_dataloader, torch.nn.BCELoss(), 0.5)

Test F1 (micro) 0.8537315167184731
Test F1 (macro) 0.7912885974039238
Test Hamming loss 0.003827209291719242


# RCV1-v2


In [267]:
#!g1.1
import gdown

url = 'https://drive.google.com/drive/folders/1itSsgk_DzDqK4Wa_YRPNWwaFFeVVu2QO?usp=sharing'
gdown.download_folder(url, quiet=True)

['/home/jupyter/work/resources/train.tgt',
 '/home/jupyter/work/resources/valid.tgt',
 '/home/jupyter/work/resources/test.src',
 '/home/jupyter/work/resources/test.tgt',
 '/home/jupyter/work/resources/train.src',
 '/home/jupyter/work/resources/train.tgt',
 '/home/jupyter/work/resources/valid.src',
 '/home/jupyter/work/resources/valid.tgt']

In [269]:
#!g1.1
res = {}
with open('rcv1.taxonomy') as f:
    for line in f:
        tmp = line.strip().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in res:
                res[tmp[i]] = []
            if i != 0:
                res[tmp[0]].append(tmp[i])
                
key2i = {}
for i, elem in enumerate(list(res.keys())):
    key2i[elem] = i

N = len(key2i) - 1

In [270]:
#!g1.1
N

103

In [273]:
#!g1.1
from tqdm import tqdm

with open('train.tgt', 'r') as f:
    with open('train.src', 'r') as g:
        with open('train_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                label = ['0'] * N
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()] - 1] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 20833/20833 [00:00<00:00, 249470.21it/s]


In [274]:
#!g1.1
from tqdm import tqdm

with open('valid.tgt', 'r') as f:
    with open('valid.src', 'r') as g:
        with open('valid_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                label = ['0'] * N
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()] - 1] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 2316/2316 [00:00<00:00, 248255.98it/s]


In [275]:
#!g1.1
with open('test.tgt', 'r') as f:
    with open('test.src', 'r') as g:
        with open('test_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                label = ['0'] * N
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()] - 1] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 781265/781265 [00:03<00:00, 230403.21it/s]


In [283]:
#!g1.1
train_dataset = RCVDataset('./train_docbert.rcv')
val_dataset = RCVDataset('./valid_docbert.rcv')
test_dataset = RCVDataset('./test_docbert.rcv')

In [284]:
#!g1.1
BATCH_SIZE = 16 # !DocBERT
N_CLASSES = test_dataset[0][1].shape[0]
BATCH_SIZE, N_CLASSES

(16, 103)

In [285]:
#!g1.1
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [286]:
#!g1.1
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [297]:
#!g1.1
!wc -l HiAGM/data/test.rcv

781265 HiAGM/data/test.rcv


In [303]:
#!g1.1
from helper.configure import Configure
config = Configure(config_json_file='HiAGM/config/rcv1-v2-Copy3.json')

INFO:  CONFIGURE: {'dataset': 'rcv1', 'data_dir': 'HiAGM/data', 'train_file': 'train.rcv', 'val_file': 'valid.rcv', 'test_file': 'test.rcv', 'prob_json': 'rcv1_overall_corpus_train_prob.json', 'hierarchy': 'rcv1.taxonomy'}
INFO:  CONFIGURE: {'dir': 'vocab', 'vocab_dict': 'word.dict', 'max_token_vocab': 60000, 'label_dict': 'label.dict'}
INFO:  CONFIGURE: {'dimension': 300, 'type': 'pretrain', 'pretrained_file': 'glove.6B/glove.6B.300d.txt', 'dropout': 0.5, 'init_type': 'uniform'}
INFO:  CONFIGURE: {'dimension': 768, 'type': 'random', 'dropout': 0.5, 'init_type': 'kaiming_uniform'}
INFO:  CONFIGURE: {'token': <helper.configure.Configure object at 0x7f80443e94f0>, 'label': <helper.configure.Configure object at 0x7f80443e9100>}
INFO:  CONFIGURE: {'bidirectional': True, 'num_layers': 1, 'type': 'GRU', 'hidden_dimension': 64, 'dropout': 0.1}
INFO:  CONFIGURE: {'kernel_size': [2, 3, 4], 'num_kernel': 100}
INFO:  CONFIGURE: {'max_length': 256, 'RNN': <helper.configure.Configure object at 0x7f

In [304]:
#!g1.1
from data_modules.vocab import Vocab
vcb = Vocab(config, min_freq=5, max_size=50000)

400000it [00:07, 56544.00it/s]
20833it [00:01, 16258.62it/s]
2316it [00:00, 15779.25it/s]
781265it [00:51, 15056.37it/s]


INFO:  Building Vocabulary....
INFO:  Generating Vocabulary from Corpus...
INFO:  Loading vocabulary from pretrained embedding...
INFO:  Loading TRAIN subset...
INFO:  Loading VAL subset...
INFO:  Loading TEST subset...
INFO:  Vocabulary of token 609022
INFO:  Vocabulary of label 103
INFO:  Shrinking Vocabulary...
INFO:  Shrinking Vocabulary of tokens: 50000
INFO:  Vocabulary of token with the size of 50002
INFO:  Save Vocabulary in vocab/word.dict
INFO:  Vocabulary of label with the size of 103
INFO:  Save Vocabulary in vocab/label.dict


In [305]:
#!g1.1
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 2.333 Gb


In [306]:
#!g1.1
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats('cuda')
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 2.333 Gb


In [307]:
#!g1.1
model = SEDocBert(bert, N_CLASSES, config, vcb).to(device)
optimizer = optim.AdamW(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE, weight_decay=0.01)

In [309]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("SEDocBERT"), wandb_iter_start=0)

Test F1 (micro) 0.791472500892555
Test F1 (macro) 0.5672662778761786
Test Hamming loss 0.011199358305653865
13030


In [311]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=13030)

Test F1 (micro) 0.7956744997408174
Test F1 (macro) 0.5939221149260359
Test Hamming loss 0.011159051628561673
19545


In [313]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=19545)

Batch train loss,██▃▁
steps_per_sec,▁█▆█
Batch train loss,0.17463
steps_per_sec,1.87496


Test F1 (micro) 0.7937055488241203
Test F1 (macro) 0.591001050176596
Test Hamming loss 0.01124027987290095
26060


In [314]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=26060)

Batch train loss,▆▅▇▅▅▃▃▄▆▆▂▄▃▄▇▅▂█▇▄▁▄▄▇▅▄▄▄▃▂▃▅▃▂▇▆▅▆▂▇
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▁▄▆▇█
Train F1 (micro),▁▃▄██
Train Hamming loss,█▆▅▁▁
Train loss,█▅▅▂▁
Validation F1 (macro),█▁▁▅▄
Validation F1 (micro),█▆▁▇▆
Validation Hamming loss,▁▄█▁▄


Test F1 (micro) 0.7980454854245786
Test F1 (macro) 0.5949713251385749
Test Hamming loss 0.010919181724259914
32575


In [315]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=32575)

Batch train loss,▂▃▁▃▃▅▂▃▂▃▅▅▅▃▆▅█▄▄▂▃▅▃▃▂▃▃▂▃▃▄▁▃▄▄▃▂▂▅▃
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▃▅▁██
Train F1 (micro),▃▃▁▆█
Train Hamming loss,▇▇█▃▁
Train loss,▇█▇▃▁
Validation F1 (macro),▁▆▃█▅
Validation F1 (micro),▂▁▂█▆
Validation Hamming loss,▆██▁▄


Test F1 (micro) 0.7932065026417069
Test F1 (macro) 0.5938154799907283
Test Hamming loss 0.0111881522463004
39090


In [316]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=39090)

Batch train loss,▅▆▃▄▄▅▄▅▄▄▅▁▇▄▄▅▄▅▆▄▅▃▃▄▆▃▃▆▅▄▄▆▅▃▄▅█▅▄▁
Test F1 (macro),▁
Test F1 (micro),▁
Test Hamming loss,▁
Train F1 (macro),▁▃█▃▇
Train F1 (micro),▁▁▄▅█
Train Hamming loss,██▅▄▁
Train loss,█▅▃▃▁
Validation F1 (macro),▆▃▅█▁
Validation F1 (micro),▇▃▃█▁
Validation Hamming loss,▁█▆▂█


Test F1 (micro) 0.797973579511077
Test F1 (macro) 0.5981608000370828
Test Hamming loss 0.011019885673097037
45605


In [317]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, test_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 5, WandbWriter("SEDocBERT"), wandb_iter_start=45605)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pos. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


KeyboardInterrupt: 

In [ ]:
#!g1.1
